# LAB 3: NUMBA 
### SGD Optimization with Numba
### AI and Machine Learning // Suchkova Natalia М8О-114М-22
09.11.2022 @ MAI IT-Center

<a id='gen0'></a>
### The Task
Выполнить реализацию SGD в Numba и сравнить по времени с реализацией в Лабораторной Работе №1

## Table of Contents

1. [**Without Numba**](#gen1)

    - [Classical GD](#gen11) 
        + [Himmelblau](#gen111)
        + [Rastrigin](#gen112)
        + [Rosenbrock](#gen113)

    - [AdaGrad](#gen12)
        + [Himmelblau](#gen121)
        + [Rastrigin](#gen122)
        + [Rosenbrock](#gen123)


2. [**With Numba**](#gen2)

    - [Classical GD](#gen21) 
        + [Himmelblau](#gen211)
        + [Rastrigin](#gen212)
        + [Rosenbrock](#gen213)

    - [AdaGrad](#gen22)
        + [Himmelblau](#gen221)
        + [Rastrigin](#gen222)
        + [Rosenbrock](#gen223)



3. [**Result Comparison Table**](#gen3)



In [282]:
import numpy as np
import pandas as pd
from numba import njit

from typing import Tuple, Mapping

import matplotlib.pyplot as plt
from IPython import display

In [283]:
res_df = pd.DataFrame(columns=['Problem', 'Method', 'Time w/o Numba', 'Numba Time']) # создаем дф для учета результатов

<a id='gen1'></a>

## Without Numba

In [284]:
class Himmelblau:
    
    def function(x: np.ndarray) -> np.float64:
        return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2
    
    def gradient(x: np.ndarray) -> np.array:
        return np.array([4 * x[0] * (x[0]**2 + x[1] - 11) \
                         + 2 * (x[0] + x[1]**2 - 7),
                         2 * (x[0]**2 + x[1] - 11) \
                         + 4 * x[1] * (x[0] + x[1]**2 - 7)])

class Rosenbrock: 
    
    def function(x: np.ndarray, b: int = 100) -> np.float64:
        return b * (x[1] - x[0]**2)**2 + (x[0] - 1)**2

    def gradient(x: np.ndarray, b: int = 100) -> np.array:

        return np.array([2 * (x[0] - 1) \
                         - 4 * b * x[0] * (x[1] - x[0]**2),
                         2 * b * (x[1] - x[0]**2)])
    
class Rastrigin:
    
    def function(x: np.ndarray, A: int = 10) -> np.float64:
        return list(x**2 - A * np.cos(2 * np.pi * x))[0] # + A
    
    def gradient(x: np.float32, A: int = 10) -> np.float64:
            return 2 * x + A * 2 * np.pi * np.sin(2 * np.pi * x)

Упростим еще немного код, который был.

In [285]:
# def classic_GD (
#                 function: Mapping, gradient_of_function: Mapping,
#                 start: np.ndarray, learning_rate: float = 0.01, 
#                 n_iter: int = 100, tolerance: float = 1e-5
#                 ) -> Tuple [np.ndarray, np.float32]:
    
#     """ 
#     Args:
#         function (Mapping): минимизруемая функция
#         gradient_of_function (Mapping): градиент минимизируемой функции
#         start (np.ndarray): рандомная стартовая точка
#         learning_rate (float): шаг минимизации
#         n_iter (int): количество итераций градиентного спуска
#         tolerance (float): минимальное допустимое изменение 
#                     значения минимизируемой величины
#     Return:
#         tuple with found minimum coordinates, found minimum function value, 
#         and plotting data list with multiple np.ndarray aka dots for plotting
    
#    """ 
        
#     current_point = start.copy()
#     current_point = current_point.astype('float64')

#     for iter in range(n_iter):
#         diff = learning_rate * -gradient_of_function(current_point)

#         if np.all(np.abs(diff) <= tolerance):
#             print(f'\033[1mEarly stopping!!\033[0m')
#             break

#         iter_count = iter + 1    
#         current_point += diff

#     print(f' Finished in {iter_count} iterations\n', \
#           f'Minimum point coordinates \033[1m{current_point}\033[0m,', \
#           f'with function value = \033[1m{round(function(current_point), 4)}\033[0m')


#     return current_point, function(current_point)

<a id='gen11'></a>

In [286]:
def classic_GD (
                function: Mapping, gradient_of_function: Mapping,
                start: np.ndarray, learning_rate: float = 0.01, 
                n_iter: int = 100,
                ) -> Tuple [np.ndarray, np.float32]:
    
    """ 
    Args:
        function (Mapping): минимизруемая функция
        gradient_of_function (Mapping): градиент минимизируемой функции
        start (np.ndarray): рандомная стартовая точка
        learning_rate (float): шаг минимизации
        n_iter (int): количество итераций градиентного спуска

    Return:
        tuple with found minimum coordinates, found minimum function value, 
        and plotting data list with multiple np.ndarray aka dots for plotting
    
   """    
    current_point = start.copy()
    current_point = current_point.astype('float64')

    for iter in range(n_iter):   
        current_point = current_point - learning_rate * gradient_of_function(current_point)

    return current_point, function(current_point)

In [287]:
# wall time

<a id='gen111'></a>
#### Himmelblau

In [288]:
%%time 
GD_Him = classic_GD(Himmelblau.function, Himmelblau.gradient, np.array([-100, -100]), learning_rate=0.00001, n_iter=7000)

Wall time: 71.8 ms


In [289]:
print(f'Minimum point coordinates \033[1m{GD_Him[0]}\033[0m,', \
      f'with function value = \033[1m{round(GD_Him[1], 4)}\033[0m')

Minimum point coordinates [-3.78698073 -3.29532809], with function value = 0.0073


In [290]:
# cpu time
time = %timeit -o classic_GD(Himmelblau.function, Himmelblau.gradient, np.array([-100, -100]), learning_rate=0.00001, n_iter=7000)

66.5 ms ± 306 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [291]:
data = ['Himmelblau', 'Classical GD', str(time)[:25], '']
res_df.loc[len(res_df)] = data

In [292]:
res_df

,Problem,Method,Time w/o Numba,Numba Time
0,Himmelblau,Classical GD,66.5 ms ± 306 µs per loop,


<a id='gen112'></a>
#### Rastrigin

In [293]:
%%time
GD_Rast = classic_GD(Rastrigin.function, Rastrigin.gradient, np.array([100, 100]), learning_rate=0.1, n_iter=9000)

Wall time: 71.8 ms


In [294]:
print(f'Minimum point coordinates \033[1m{GD_Rast[0]}\033[0m,', \
      f'with function value = \033[1m{round(GD_Rast[1], 4)}\033[0m')

Minimum point coordinates [0.76548588 0.76548588], with function value = -0.3855


In [295]:
# cpu time
time = %timeit -o classic_GD(Rastrigin.function, Rastrigin.gradient, np.array([100, 100]), learning_rate=0.1, n_iter=9000)

58.7 ms ± 12.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [296]:
data = ['Rastrigin', 'Classical GD', str(time)[:25], '']
res_df.loc[len(res_df)] = data

<a id='gen113'></a>
#### Rosenbrock

In [297]:
%%time
GD_Ros = classic_GD(Rosenbrock.function, Rosenbrock.gradient, np.array([10, 10]), 
                    learning_rate=0.00001, n_iter=200000)

Wall time: 817 ms


In [298]:
print(f'Minimum point coordinates \033[1m{GD_Ros[0]}\033[0m,', \
      f'with function value = \033[1m{round(GD_Ros[1], 4)}\033[0m')

Minimum point coordinates [3.02969959 9.18234049], with function value = 4.1207


In [299]:
# cpu time
time = %timeit -o classic_GD(Rosenbrock.function, Rosenbrock.gradient, np.array([10, 10]), learning_rate=0.00001, n_iter=200000)

772 ms ± 3.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [300]:
data = ['Rosenbrock', 'Classical GD', str(time)[:25], '']
res_df.loc[len(res_df)] = data

<a id='gen12'></a>

In [301]:
def GD_AdaGrad(
            function: Mapping, gradient: Mapping, start: np.ndarray,
            gamma: np.float64 = 0.1, n_iter: int = 100, 
            learning_rate: float = 0.01, tolerance=1e-06, 
            dtype="float64", rand_state: int = 12
            ) -> Tuple [np.ndarray, np.ndarray]:
    """ 
        function (Mapping):  минимизруемая функция
        gradient (Mapping): градиент заданной выше функции
        start (np.ndarray): рандомная стартовая точка/точки
        gamma (float): скорость затухания скользащих средних ф-ии потерь
        n_iter (int): количество итераций градиентного спуска
        learning_rate (float): шаг минимизации
        tolerance (float): минимальное допустимое изменение 
                           значения минимизируемой величины
        dtype (str): тип данных
        rand_state (int): зерно рандомайзера
        
    """
    dtype_ = np.dtype(dtype)

    cur_point = start.copy()
    cur_point = cur_point.astype('float64')

    G = np.zeros(cur_point.shape, dtype=dtype_) 
   
    for iter in range(n_iter):
        loss = gradient(cur_point)
        G = gamma * G + loss ** 2

        if np.all(np.abs(loss/np.sqrt(G)) <= tolerance):
            print(f'\033[1mEarly stopping!!\033[0m')
            break

        cur_point -= (loss/np.sqrt(G)) * learning_rate

    return cur_point, function(cur_point)

<a id='gen121'></a>
#### Himmelblau

In [302]:
%%time
Him_Ada = GD_AdaGrad(Himmelblau.function, Himmelblau.gradient, np.array([100, 100]),
                     gamma=0.8, learning_rate=0.01, n_iter=30000)

Wall time: 515 ms


In [303]:
print(f'Minimum point coordinates \033[1m{Him_Ada[0]}\033[0m,', 
      f'with function value = \033[1m{round(Him_Ada[1], 6)}\033[0m')

Minimum point coordinates [3.00223411 2.00223281], with function value = 0.000369


In [304]:
# cpu time
time = %timeit -o GD_AdaGrad(Himmelblau.function, Himmelblau.gradient, np.array([100, 100]), gamma=0.8, learning_rate=0.01, n_iter=30000)

494 ms ± 6.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [305]:
data = ['Himmelblau', 'AdaGrad', str(time)[:25], '']
res_df.loc[len(res_df)] = data

<a id='gen122'></a>
#### Rastrigin

In [306]:
%%time
Rast_Ada = GD_AdaGrad(Rastrigin.function, Rastrigin.gradient, np.array([-10, 100]), 
                      tolerance=1e-4, gamma=0.2, learning_rate=0.51, n_iter=2000000)

Wall time: 30.1 s


In [307]:
print(f'Minimum point coordinates \033[1m{Rast_Ada[0]}\033[0m,', 
      f'with function value = \033[1m{round(Rast_Ada[1], 6)}\033[0m')

Minimum point coordinates [-5.02130232  5.0292553 ], with function value = 15.302918


In [308]:
# cpu time
time = %timeit -o GD_AdaGrad(Rastrigin.function, Rastrigin.gradient, np.array([-10, 100]), tolerance=1e-4, gamma=0.2, learning_rate=0.51, n_iter=2000000)

30.3 s ± 379 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [309]:
data = ['Rastrigin', 'AdaGrad', str(time)[:25], '']
res_df.loc[len(res_df)] = data

<a id='gen123'></a>
#### Rosenbrock

In [310]:
%%time
Ros_Ada = GD_AdaGrad(Rosenbrock.function, Rosenbrock.gradient, np.array([-10, -10]),
                     gamma=0.9, tolerance = 1e-3, learning_rate=0.01, n_iter=30000)

Wall time: 463 ms


In [311]:
print(f'Minimum point coordinates \033[1m{Ros_Ada[0]}\033[0m,', 
      f'with function value = \033[1m{round(Ros_Ada[1], 6)}\033[0m')

Minimum point coordinates [1.00008264 0.9954266 ], with function value = 0.002246


In [312]:
# cpu time
time = %timeit -o GD_AdaGrad(Rosenbrock.function, Rosenbrock.gradient, np.array([-10, -10]), gamma=0.9, tolerance = 1e-3, learning_rate=0.01, n_iter=30000)

459 ms ± 14.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [313]:
data = ['Rosenbrock', 'AdaGrad', str(time)[:25], '']
res_df.loc[len(res_df)] = data

In [314]:
res_df.set_index('Problem', drop=True, inplace=True)

In [315]:
res_df

,Method,Time w/o Numba,Numba Time
Problem,,,
Himmelblau,Classical GD,66.5 ms ± 306 µs per loop,
Rastrigin,Classical GD,58.7 ms ± 12.6 ms per loo,
Rosenbrock,Classical GD,772 ms ± 3.53 ms per loop,
Himmelblau,AdaGrad,494 ms ± 6.75 ms per loop,
Rastrigin,AdaGrad,30.3 s ± 379 ms per loop,
Rosenbrock,AdaGrad,459 ms ± 14.8 ms per loop,


<a id='gen2'></a>

## With Numba

In [316]:
@njit(fastmath=True)
def Himmelblau(x: np.ndarray) -> np.float64:
    return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2
@njit(fastmath=True)
def Him_grad(x: np.ndarray) -> np.array:
    return np.array([4 * x[0] * (x[0]**2 + x[1] - 11) \
                     + 2 * (x[0] + x[1]**2 - 7),
                     2 * (x[0]**2 + x[1] - 11) \
                         + 4 * x[1] * (x[0] + x[1]**2 - 7)])
@njit(fastmath=True)
def Rosenbrock(x: np.ndarray, b: int = 100) -> np.float64:
    return b * (x[1] - x[0]**2)**2 + (x[0] - 1)**2
@njit(fastmath=True)
def Rosen_grad(x: np.ndarray, b: int = 100) -> np.array:
    return np.array([2 * (x[0] - 1) \
                     - 4 * b * x[0] * (x[1] - x[0]**2),
                         2 * b * (x[1] - x[0]**2)])
@njit(fastmath=True)
def Rastrigin(x: np.ndarray, A: int = 10) -> np.float64:
    return list(x**2 - A * np.cos(2 * np.pi * x))[0] # + A
@njit(fastmath=True)
def Rast_grad(x: np.float32, A: int = 10) -> np.float64:
        return 2 * x + A * 2 * np.pi * np.sin(2 * np.pi * x)

<a id='gen21'></a>

In [317]:
@njit
def classic_GD (
                function: Mapping, gradient_of_function: Mapping,
                start: np.ndarray, learning_rate: float = 0.01, 
                n_iter: int = 100
                ) -> Tuple [np.ndarray, np.float32]:
    
    """ 
    Args:
        function (Mapping): минимизруемая функция
        gradient_of_function (Mapping): градиент минимизируемой функции
        start (np.ndarray): рандомная стартовая точка
        learning_rate (float): шаг минимизации
        n_iter (int): количество итераций градиентного спуска
    Return:
        tuple with found minimum coordinates, found minimum function value, 
        and plotting data list with multiple np.ndarray aka dots for plotting
    
   """   
    current_point = start.copy()
    current_point = current_point.astype('float64')
    
    for iter in range(n_iter):   
        current_point = current_point - learning_rate * gradient_of_function(current_point)

    return current_point, function(current_point)

<a id='gen211'></a>
#### Himmelblau

In [318]:
%%time
GD_Him_GPU = classic_GD(Himmelblau, Him_grad, np.array([-100, -100]), learning_rate=0.00001, n_iter=9500)

Wall time: 622 ms


In [319]:
print(f'Minimum point coordinates \033[1m{GD_Him_GPU[0]}\033[0m,')
print(f'with function value = \033[1m{round(GD_Him_GPU[1], 4)}\033[0m')

Minimum point coordinates [-3.78059997 -3.28525589],
with function value = 0.0002


In [320]:
# cpu time
time = %timeit -o classic_GD(Himmelblau, Him_grad, np.array([-100, -100]), learning_rate=0.00001, n_iter=9500)

1.65 ms ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [321]:
res_df.at['Himmelblau', 'Numba Time'] = str(time)[:25]

<a id='gen212'></a>
#### Rastrigin

In [322]:
%%time
GD_Rast_GPU = classic_GD(Rastrigin, Rast_grad, np.array([10, 10]), learning_rate=0.1, n_iter=200)

Wall time: 771 ms


In [323]:
print(f'Minimum point coordinates \033[1m{GD_Rast_GPU[0]}\033[0m,')
print(f'with function value = \033[1m{round(GD_Rast_GPU[1], 4)}\033[0m')

Minimum point coordinates [-1.78314385 -1.78314385],
with function value = 1.1121


In [324]:
# cpu time
time = %timeit -o classic_GD(Rastrigin, Rast_grad, np.array([10, 10]), learning_rate=0.1, n_iter=200)

64.3 µs ± 873 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [325]:
res_df.at['Rastrigin', 'Numba Time'] = str(time)[:25]

<a id='gen213'></a>
#### Rosenbrock

In [326]:
%%time
GD_Ros_GPU = classic_GD(Rosenbrock, Rosen_grad, np.array([10, 10]), learning_rate=0.00001, n_iter=200000)

Wall time: 446 ms


In [327]:
print(f'Minimum point coordinates \033[1m{GD_Ros_GPU[0]}\033[0m,')
print(f'with function value = \033[1m{round(GD_Ros_GPU[1], 4)}\033[0m')

Minimum point coordinates [3.02969959 9.18234049],
with function value = 4.1207


In [328]:
# cpu time
time = %timeit -o classic_GD(Rosenbrock, Rosen_grad, np.array([10, 10]), learning_rate=0.00001, n_iter=200000)

34 ms ± 273 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [329]:
res_df.at['Rosenbrock', 'Numba Time'] = str(time)[:25]

<a id='gen22'></a>

In [330]:
@njit
def GD_AdaGrad (
                function: Mapping, gradient: Mapping,
                start: np.ndarray, learning_rate: float = 0.01, 
                n_iter: int = 100, tolerance: float = 1e-5,
                gamma: np.float64 = 0.1, dtype="float64", 
                rand_state: int = 12
                ) -> Tuple [np.ndarray, np.float64]:
    
    """ 
    Args:
        function (Mapping):  минимизруемая функция
        gradient (Mapping): градиент заданной выше функции
        start (np.ndarray): рандомная стартовая точка/точки
        gamma (float): скорость затухания скользащих средних ф-ии потерь
        n_iter (int): количество итераций градиентного спуска
        learning_rate (float): шаг минимизации
        tolerance (float): минимальное допустимое изменение 
                           значения минимизируемой величины
        dtype (str): тип данных
        rand_state (int): зерно рандомайзера
    Return: 
        tuple with found minimum point coordinates, 
        funcation value at this point
        
   """  
    cur_point = start.copy()
    cur_point = cur_point.astype('float64')
    
    G = np.zeros(cur_point.shape, dtype=np.float64)
    
    for iter in range(n_iter):
        loss = gradient(cur_point)
        G = gamma * G + loss ** 2

        if np.all(np.abs(loss/np.sqrt(G)) <= tolerance):
            print(f'\033[1mEarly stopping!!\033[0m')
            break

        cur_point -= (loss/np.sqrt(G)) * learning_rate

    return cur_point, function(cur_point)

<a id='gen221'></a>
#### Himmelblau

In [331]:
%%time
Him_Ada = GD_AdaGrad(Himmelblau, Him_grad, np.array([100, 100]),
                     gamma=0.8, learning_rate=0.01, n_iter=30000)

Wall time: 395 ms


In [332]:
print(f'Minimum point coordinates \033[1m{Him_Ada[0]}\033[0m,', 
      f'with function value = \033[1m{round(Him_Ada[1], 6)}\033[0m')

Minimum point coordinates [3.00223411 2.00223281], with function value = 0.000369


In [333]:
# cpu time
time = %timeit -o GD_AdaGrad(Himmelblau, Him_grad, np.array([100, 100]), gamma=0.8, learning_rate=0.01, n_iter=30000)

11.1 ms ± 158 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [334]:
res_df.iat[3, 2] = str(time)[:25]

<a id='gen222'></a>
#### Rastrigin

In [335]:
%%time
Rast_Ada = GD_AdaGrad(Rastrigin, Rast_grad, np.array([-10, 100]), 
                      tolerance=1e-4, gamma=0.2, learning_rate=0.51, n_iter=2000000)

Wall time: 1.37 s


In [336]:
print(f'Minimum point coordinates \033[1m{Rast_Ada[0]}\033[0m,', 
      f'with function value = \033[1m{round(Rast_Ada[1], 6)}\033[0m')

Minimum point coordinates [-4.7446343  19.65106439], with function value = 22.848628


In [337]:
# cpu time
time = %timeit -o GD_AdaGrad(Rastrigin, Rast_grad, np.array([-10, 100]), tolerance=1e-4, gamma=0.2, learning_rate=0.51, n_iter=2000000)

819 ms ± 25.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [338]:
res_df.iat[4, 2] = str(time)[:25]

<a id='gen223'></a>
#### Rosenbrock

In [339]:
%%time
Ros_Ada = GD_AdaGrad(Rosenbrock, Rosen_grad, np.array([-10, -10]),
                     gamma=0.9, tolerance = 1e-3, learning_rate=0.01, n_iter=30000)

Wall time: 586 ms


In [340]:
print(f'Minimum point coordinates \033[1m{Ros_Ada[0]}\033[0m,', 
      f'with function value = \033[1m{round(Ros_Ada[1], 6)}\033[0m')

Minimum point coordinates [1.00008264 0.9954266 ], with function value = 0.002246


In [341]:
# cpu time
time = %timeit -o GD_AdaGrad(Rosenbrock, Rosen_grad, np.array([-10, -10]), gamma=0.9, tolerance = 1e-3, learning_rate=0.01, n_iter=30000)

11.5 ms ± 247 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [342]:
res_df.iat[5, 2] = str(time)[:25]

<a id='gen3'></a>
## Results

In [343]:
res_df

,Method,Time w/o Numba,Numba Time
Problem,,,
Himmelblau,Classical GD,66.5 ms ± 306 µs per loop,1.65 ms ± 17.3 µs per loo
Rastrigin,Classical GD,58.7 ms ± 12.6 ms per loo,64.3 µs ± 873 ns per loop
Rosenbrock,Classical GD,772 ms ± 3.53 ms per loop,34 ms ± 273 µs per loop (
Himmelblau,AdaGrad,494 ms ± 6.75 ms per loop,11.1 ms ± 158 µs per loop
Rastrigin,AdaGrad,30.3 s ± 379 ms per loop,819 ms ± 25.9 ms per loop
Rosenbrock,AdaGrad,459 ms ± 14.8 ms per loop,11.5 ms ± 247 µs per loop


Видим, что нумба сильно ускоряет результаты. Посчитаем точно во сколько раз.

In [350]:
wo_N = []
w_N = []
for i in res_df['Time w/o Numba']:
    cont = i.split()[:2]
    wo_N.append(cont)
for t in wo_N:
    t[0] = float(t[0])
    if t[1] == 's':  # переводим все в микросекунды
        t[0] = t[0] * 1e+6
    elif t[1] == 'ms':
        t[0] = t[0] * 1e+3
        
for i in res_df['Numba Time']:
    cont = i.split()[:2]
    w_N.append(cont)
for t in w_N:
    t[0] = float(t[0])
    if t[1] == 's':  # переводим все в микросекунды
        t[0] = t[0] * 1e+6
    elif t[1] == 'ms':
        t[0] = t[0] * 1e+3


In [351]:
# рассчитаем во сколько раз нумбa быстрее для каждого случая
i = 0  
boost = []
while i < len(w_N):
    boost.append(round(wo_N[i][0]/w_N[i][0], 2))
    i += 1

In [352]:
res_df['Time Boost (times)'] = boost

In [353]:
res_df

,Method,Time w/o Numba,Numba Time,Time Boost (times)
Problem,,,,
Himmelblau,Classical GD,66.5 ms ± 306 µs per loop,1.65 ms ± 17.3 µs per loo,40.30
Rastrigin,Classical GD,58.7 ms ± 12.6 ms per loo,64.3 µs ± 873 ns per loop,912.91
Rosenbrock,Classical GD,772 ms ± 3.53 ms per loop,34 ms ± 273 µs per loop (,22.71
Himmelblau,AdaGrad,494 ms ± 6.75 ms per loop,11.1 ms ± 158 µs per loop,44.50
Rastrigin,AdaGrad,30.3 s ± 379 ms per loop,819 ms ± 25.9 ms per loop,37.00
Rosenbrock,AdaGrad,459 ms ± 14.8 ms per loop,11.5 ms ± 247 µs per loop,39.91


[Go Up](#gen0)